# **Day 6: Wait For It**

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring my kernel:

In [1]:
# Changing the current working directory
%cd ..

# Enabling the autoreload extension
%load_ext autoreload
%autoreload 2

/Users/thubbard/Documents/Personal/Programming/advent-of-code-2023


Now, I'm going to import some libraries:

In [2]:
# Import statements
import pandas as pd
import re

Finally, I'll load in the data for this puzzle. 

In [3]:
# Load in the data for the puzzle
with open("data/input-files/day-06-input.txt", "r") as txt_file:
    input_data = txt_file.readlines()

# Parsing Input Data
Let's parse the input data below:

In [22]:
# Parse out each line into the parsed_info dict
parsed_info = {}
for line in input_data:
    split_line = re.sub(" +", " ", line.strip()).split(" ")
    data_type = split_line[0].split(":")[0].lower()
    records = [int(x) for x in split_line[1:]]
    parsed_info[data_type] = records

# Now, make a DataFrame containing the information about each race
race_info_df_records = []
for idx in range(len(list(parsed_info.values())[0])):
    cur_race_info = {
        "race_num": idx
    }
    for key in parsed_info:
        cur_race_info[key] = parsed_info[key][idx]
    race_info_df_records.append(cur_race_info)
race_info_df = pd.DataFrame.from_records(race_info_df_records)

# Show off the DataFrame we've created
race_info_df

,race_num,time,distance
0,0,48,296
1,1,93,1928
2,2,85,1236
3,3,95,1391


# Determining Strategies
Next: I need to figure out a way to determine a strategy for each of the races. 

In [25]:
def determine_race_strategies(alotted_time, best_distance):
    """
    This method will generate a DataFrame containing all of the different "strategies" for a
    particular race. Each strategy will be a single row, where `hold_time` represents the number of
    seconds you hold down the button, and `resulting_distance` represents the distance that
    your boat travels during the rest of the `alotted_time`. If you beat the `best_distance`, then
    the `new_record` column will be `True`; otherwise, it will be `False`.
    """

    # Iterate through the different numbers of seconds you can hold the button
    race_strategies_df_records = []
    for cur_hold_time in range(alotted_time + 1):
        # Calculate all of the relevant information for this strategy
        remaining_time_after_hold = alotted_time - cur_hold_time
        speed_during_remaining_hold_time = cur_hold_time
        total_distance_travelled = (
            speed_during_remaining_hold_time * remaining_time_after_hold
        )
        new_record = total_distance_travelled > best_distance

        # Add the information to the race_strategies_df_records
        race_strategies_df_records.append(
            {
                "hold_time": cur_hold_time,
                "time_spent_moving": remaining_time_after_hold,
                "speed_during_movement": speed_during_remaining_hold_time,
                "total_distance_travelled": total_distance_travelled,
                "new_record": new_record,
            }
        )

    # Create a DataFrame from the race_strategies_df_records, and then return it
    return pd.DataFrame.from_records(race_strategies_df_records)

With this method in hand, we can calculate the number of winning strategies for each of the races:

In [32]:
# We're going to make a new DataFrame that contains a column enumerating the winning strategies 
winning_strategies_df = race_info_df.copy()
n_winning_strategies_col_values = []
for row in winning_strategies_df.itertuples():
    cur_race_strategies_df = determine_race_strategies(row.time, row.distance)
    n_winning_strategies_col_values.append(
        cur_race_strategies_df["new_record"].sum()
    )
winning_strategies_df["n_winning_strategies"] = n_winning_strategies_col_values

# Show off the DataFrame with this new column
winning_strategies_df

,race_num,time,distance,n_winning_strategies
0,0,48,296,33
1,1,93,1928,30
2,2,85,1236,48
3,3,95,1391,58


Finally, we can answer the first question by multiplying the number of winning strategies for each race:

In [35]:
# Print out the product of all of the numbers of winning strategies
final_product = 1
for n_winning_strategy in winning_strategies_df["n_winning_strategies"]:
    final_product *= n_winning_strategy
print(f"In total, there are '{final_product}' ways to win the grand prize.")

In total, there are '2756160' ways to win the grand prize.
